In [ ]:
# R code for all event level models of Analysis II: Driving Style Parameters

In [2]:
# Load some packages
library(reshape2)
library(ggplot2)
library(cowplot)
library(ez)
library(nlme)
library(multcomp)
library(pastecs)
theme_set(theme_cowplot(font_size=8))
library(usdm)
library(r2glmm)
library(car)
library(MASS)
library(MCMCglmm)
library(lmerTest)
library(expss)
library(moments)

In [15]:
# Read signals, extract directions and distance thresholds
signals = read.csv("../metrics/all_events.csv")
colnames(signals)

# Asign direction columns for acceleration and jerk (positive for >= 0 and negative for < 0)
signals$pos_latA <- signals$MaxLatAcc >= 0
signals$pos_longA <- signals$MaxLongAcc >= 0

signals$pos_latJ <- signals$MaxLatJerk >= 0
signals$pos_longJ <- signals$MaxLongJerk >= 0

# Thresholds for the presence of a lead vehicle
signals$present_stop <- signals$LongDist < 190
signals$present_pass <- signals$LatDist < 190

[1] "pid"          "LatDist"      "LongDist"     "MaxVel"       "MeanVel"     
 [6] "MaxLatAcc"    "MeanLatAcc"   "MaxLongAcc"   "MeanLongAcc"  "MaxLatJerk"  
[11] "MeanLatJerk"  "MaxLongJerk"  "MeanLongJerk" "NumPks"       "MaxPkAmp"    
[16] "MeanScl"      "MeanEntropy"  "MaxEntropy"   "MeanDwell"    "MaxDwell"    
[21] "MeanHr"       "MaxHr"        "Event"        "Trial"

In [10]:
# Max-valued regressions

# column indexes for the response/dependent variables of interest (NumPks, MaxPkAmp, MaxHR --
# MaxEntropy is handled separately due to missing data in signals)
response_ids = c(14, 15, 22)
responses = colnames(signals)[response_ids]

#----------------------------- Models for all main effects -----------------------------#
main_max = lapply(responses, function(x) {
    lme(
        eval(
            substitute(
                resp ~ MaxLongAcc*pos_longA + MaxLatAcc*pos_latA + 
                    MaxLongJerk*pos_longJ + MaxLatJerk*pos_latJ +
                    present_pass,
                list(resp = as.name(x))
            )
        ),
        random = ~1|pid/Trial,
        correlation = corAR1(0, form=~1|pid/Trial),
        data = signals,
        method = "ML"
    )
})

entropy_main_max = lme(
    MaxEntropy ~ present_stop +
                MaxLongAcc*pos_longA + MaxLatAcc*pos_latA + 
                MaxLongJerk*pos_longJ + MaxLatJerk*pos_latJ +
                present_pass,
    
    random = ~1|pid/Trial,
    correlation = corAR1(0, form=~1|pid/Trial),
    data = na.omit(signals),
    method = "ML"
)

#-------------------------- Models for all interaction effects ---------------------------#
inter_max = lapply(responses, function(x) {
    lme(
        eval(
            substitute(
                resp ~ present_stop +
                    MaxLongAcc*pos_longA + MaxLatAcc*pos_latA + 
                    MaxLongJerk*pos_longJ + MaxLatJerk*pos_latJ +
                    present_stop*MaxLongAcc*pos_longA + present_stop*MaxLongJerk*pos_longJ +
                    present_pass,
                list(resp = as.name(x))
            )
        ),
        random = ~1|pid/Trial,
        correlation = corAR1(0, form=~1|pid/Trial),
        data = signals,
        method = "ML"
    )
})

entropy_inter_max = lme(
    MaxEntropy ~ present_stop +
                MaxLongAcc*pos_longA + MaxLatAcc*pos_latA +
                MaxLongJerk*pos_longJ + MaxLatJerk*pos_latJ +
                present_stop*MaxLongAcc*pos_longA + present_stop*MaxLongJerk*pos_longJ +
                present_pass,
    
    random = ~1|pid/Trial,
    correlation = corAR1(0, form=~1|pid/Trial),
    data = na.omit(signals),
    method = "ML"
)

In [11]:
# Mean-valued regressions

# column indexes for the response/dependent variables of interest (Mean SCL and Mean HR 
# -- MeanEntropy is handled separately due to missing data in signals
response_ids = c(16, 21)
responses = colnames(signals)[response_ids]

#---------------------------- Models for all main effects -----------------------------#
main_mean = lapply(responses, function(x) {
    lme(
        eval(
            substitute(
                resp ~ present_stop + 
                    MeanLongAcc + MeanLatAcc + MeanLongJerk + MeanLatJerk + 
                    present_pass,
                list(resp = as.name(x))
            )
        ),
        random = ~1|pid/Trial,
        correlation = corAR1(0, form=~1|pid/Trial),
        data = signals,
        method = "ML"
    )
})

entropy_main_mean = lme(
    MeanEntropy ~ present_stop + 
                MeanLongAcc + MeanLatAcc + MeanLongJerk + MeanLatJerk + 
                present_pass,
    
    random = ~1|pid/Trial,
    correlation = corAR1(0, form=~1|pid/Trial),
    data = na.omit(signals),
    method = "ML"
)

#---------------------- Models for all interaction effects --------------------------#
inter_mean = lapply(responses, function(x) {
    lme(
        eval(
            substitute(
                resp ~ present_stop + 
                    MeanLongAcc + MeanLatAcc + MeanLongJerk + MeanLatJerk +
                    present_stop:MeanLongAcc + present_stop:MeanLongJerk +
                    present_pass,
                list(resp = as.name(x))
            )
        ),
        random = ~1|pid/Trial,
        correlation = corAR1(0, form=~1|pid/Trial),
        data = signals,
        method = "ML"
    )
})

entropy_inter_mean = lme(
    MeanEntropy ~ present_stop + 
                MeanLongAcc + MeanLatAcc + MeanLongJerk + MeanLatJerk + 
                present_stop:MeanLongAcc + present_stop:MeanLongJerk +
                present_pass,
    
    random = ~1|pid/Trial,
    correlation = corAR1(0, form=~1|pid/Trial),
    data = na.omit(signals),
    method = "ML"
)

In [12]:
# Combine all main effect models and interaction effect models (excluding entropy 
# which we process separately)
main_eff_models = c(main_max, main_mean)
inter_eff_models = c(inter_max, inter_mean)

response_names = c(
    "NUMBER OF PEAKS",
    "MAX PEAK AMPLITUDE", 
    "MAX HR", 
    "MEAN SCL", 
    "MEAN HR"
)
model_idx  = c(1:5)

In [13]:
cat("ALL MAIN EFFECTS\n")
cat("-------------------------------------------------------------------------------\n")

model_print = function(x, y, z){
    cat(y[x])
    cat("\n")
    print(summary(z[[x]]))
    cat("\n\n\n")
    cat("==============================================================================
    \n\n\n")
}

lapply(model_idx, model_print, y=response_names, z=main_eff_models)

cat("MAXIMUM ENTROPY\n")
print(summary(entropy_main_max))
cat("\n================================================================================
\n\n\n")

cat("MEAN ENTROPY\n")
print(summary(entropy_main_mean))
cat("=================================================================================\n")

ALL MAIN EFFECTS
-------------------------------------------------------------------------------
NUMBER OF PEAKS
Linear mixed-effects model fit by maximum likelihood
 Data: signals 
       AIC     BIC    logLik
  993.3729 1060.22 -478.6865

Random effects:
 Formula: ~1 | pid
        (Intercept)
StdDev:   0.5439833

 Formula: ~1 | Trial %in% pid
        (Intercept) Residual
StdDev:   0.3997903 1.060422

Correlation Structure: AR(1)
 Formula: ~1 | pid/Trial 
 Parameter estimate(s):
       Phi 
-0.2227402 
Fixed effects: eval(substitute(resp ~ MaxLongAcc * pos_longA + MaxLatAcc * pos_latA +      MaxLongJerk * pos_longJ + MaxLatJerk * pos_latJ + present_pass,      list(resp = as.name(x)))) 
                              Value Std.Error  DF   t-value p-value
(Intercept)               -0.175494  0.646571 213 -0.271423  0.7863
MaxLongAcc                -0.288474  0.197552 213 -1.460240  0.1457
pos_longATRUE             -7.296330  4.914970 213 -1.484512  0.1392
MaxLatAcc                  2.175

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

MAXIMUM ENTROPY
Linear mixed-effects model fit by maximum likelihood
 Data: na.omit(signals) 
        AIC       BIC   logLik
  -809.6656 -743.6128 423.8328

Random effects:
 Formula: ~1 | pid
        (Intercept)
StdDev:  0.03449753

 Formula: ~1 | Trial %in% pid
        (Intercept)   Residual
StdDev: 0.005433391 0.03814107

Correlation Structure: AR(1)
 Formula: ~1 | pid/Trial 
 Parameter estimate(s):
      Phi 
0.2031746 
Fixed effects: MaxEntropy ~ present_stop + MaxLongAcc * pos_longA + MaxLatAcc *      pos_latA + MaxLongJerk * pos_longJ + MaxLatJerk * pos_latJ +      present_pass 
                                Value  Std.Error  DF   t-value p-value
(Intercept)                0.21079054 0.02761537 164  7.633088  0.0000
present_stopTRUE          -0.00101904 0.00898122 164 -0.113464  0.9098
MaxLongAcc                -0.01104546 0.00867029 164 -1.273944  0.2045
pos_longATRUE             -0.09492463 0.16464663 164 -0.576536  0.5650
MaxLatAcc                  0.02455950 0.04798390 164 

In [14]:
cat("ALL INTERACTION EFFECTS\n")
cat("---------------------------------------------------------------------------------\n")

model_print = function(x, y, z){
    cat(y[x])
    cat("\n")
    print(summary(z[[x]]))
    cat("\n\n\n")
    cat("================================================================================
    \n\n\n")
}

lapply(model_idx, model_print, y=response_names, z=inter_eff_models)

cat("MAXIMUM ENTROPY\n")
print(summary(entropy_inter_max))
cat("\n==================================================================================
\n\n\n")

cat("MEAN ENTROPY\n")
print(summary(entropy_inter_mean))
cat("====================================================================================
\n")

ALL INTERACTION EFFECTS
---------------------------------------------------------------------------------
NUMBER OF PEAKS
Linear mixed-effects model fit by maximum likelihood
 Data: signals 
       AIC      BIC    logLik
  980.2035 1073.047 -465.1017

Random effects:
 Formula: ~1 | pid
        (Intercept)
StdDev:   0.5290276

 Formula: ~1 | Trial %in% pid
        (Intercept) Residual
StdDev:    0.333227 1.023208

Correlation Structure: AR(1)
 Formula: ~1 | pid/Trial 
 Parameter estimate(s):
       Phi 
-0.1674819 
Fixed effects: eval(substitute(resp ~ present_stop + MaxLongAcc * pos_longA +      MaxLatAcc * pos_latA + MaxLongJerk * pos_longJ + MaxLatJerk *      pos_latJ + present_stop * MaxLongAcc * pos_longA + present_stop *      MaxLongJerk * pos_longJ + present_pass, list(resp = as.name(x)))) 
                                                Value Std.Error  DF    t-value
(Intercept)                                  0.451871  0.873122 206  0.5175342
present_stopTRUE                  

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

MAXIMUM ENTROPY
Linear mixed-effects model fit by maximum likelihood
 Data: na.omit(signals) 
        AIC       BIC   logLik
  -807.9931 -721.0815 428.9965

Random effects:
 Formula: ~1 | pid
        (Intercept)
StdDev:  0.03449375

 Formula: ~1 | Trial %in% pid
        (Intercept)   Residual
StdDev: 0.008907801 0.03617403

Correlation Structure: AR(1)
 Formula: ~1 | pid/Trial 
 Parameter estimate(s):
      Phi 
0.1086203 
Fixed effects: MaxEntropy ~ present_stop + MaxLongAcc * pos_longA + MaxLatAcc *      pos_latA + MaxLongJerk * pos_longJ + MaxLatJerk * pos_latJ +      present_stop * MaxLongAcc * pos_longA + present_stop * MaxLongJerk *      pos_longJ + present_pass 
                                                Value Std.Error  DF   t-value
(Intercept)                                 0.2440049 0.0366615 158  6.655610
present_stopTRUE                           -0.1016080 0.0618691 158 -1.642305
MaxLongAcc                                  0.0058013 0.0114424 158  0.507001
pos_longAT





MEAN ENTROPY
Linear mixed-effects model fit by maximum likelihood
 Data: na.omit(signals) 
        AIC       BIC   logLik
  -809.0242 -763.8301 417.5121

Random effects:
 Formula: ~1 | pid
        (Intercept)
StdDev:  0.03793735

 Formula: ~1 | Trial %in% pid
        (Intercept)   Residual
StdDev:  0.01711378 0.03575379

Correlation Structure: AR(1)
 Formula: ~1 | pid/Trial 
 Parameter estimate(s):
         Phi 
-0.004047969 
Fixed effects: MeanEntropy ~ present_stop + MeanLongAcc + MeanLatAcc + MeanLongJerk +      MeanLatJerk + present_stop:MeanLongAcc + present_stop:MeanLongJerk +      present_pass 
                                    Value  Std.Error  DF   t-value p-value
(Intercept)                    0.20649339 0.03086557 170  6.690088  0.0000
present_stopTRUE              -0.01710997 0.03562358 170 -0.480299  0.6316
MeanLongAcc                   -0.01155859 0.02378168 170 -0.486029  0.6276
MeanLatAcc                    -0.00963092 0.01680161 170 -0.573214  0.5673
MeanLongJerk